# Combine languages with *language code* or language word and convert to json

In [21]:
DATA_PATH = "../../2023InflectionST/part1/data/"
OUT_DIR = "data/"
# LANGS = ["deu","eng"]
# TARGET_LANG = "deu_eng"
# LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"]
# TARGET_LANG = "mul"
LANGS_UNIQUE = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"]+["ote","pol","amh","csb"]
#TARGET_LANG = "mul_2"
LANG_WORD_OF_CODE = {"grc":"Ancient Greek",
                  "dan": "Danish",
                  "fra":"French",
                  "sme":"Sámi",
                  "deu":"German",
                  "nav":"Navajo",
                  "jap":"Japanese",
                  "klr":"Khaling",
                  "eng":"English",
                  "ote":"Mezquital Otomi",
                  "pol":"Polish",
                  "amh":"Amharic",
                  "csb":"Kashubian"}

TARGET_LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","ote","pol","amh","csb","mul"]
DATASET_COMPLETE = ["trn","tst","dev"]
LANGUAGE_CODES = [True,False]
print(OUT_DIR)
for target_lang in TARGET_LANGS:
    target_lang_initials = target_lang
    for lang_code in LANGUAGE_CODES:
        if lang_code: 
            target_lang = target_lang_initials  + "-language-code"
        else: 
            target_lang = target_lang_initials  + "-language-word"
            
        for dataset in DATASET_COMPLETE:
            if dataset != "tst" and  "csb" in target_lang: continue # csb only as test data
                
            OUT=OUT_DIR + target_lang  + "_" + dataset + ".json"
            with open (OUT,"w") as out_file:
                print(out_file)
                if target_lang in ["mul-language-code","mul-language-word"]: LANGS = LANGS_UNIQUE
                else: LANGS = [target_lang_initials]
                for lang in LANGS:
                    if dataset != "tst" and lang == "csb": continue # csb only as test data
                        
                    with open(DATA_PATH + lang + "." + dataset,"r") as in_file:
                        for line in in_file:
                            line = line.strip()
                            lemma, features, target = line.split("\t")
                            if lang_code: context = lang + ": "
                            else: context = LANG_WORD_OF_CODE[lang]+ ": "
                            json_line = f'{{"input": "{context}Inflect {lemma} | {features}","target": "{target}"}}'
                            out_file.write(json_line + "\n")



data/
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
<_io.TextIOWrapper name='data/grc-language-code_trn.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-code_tst.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-code_dev.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-word_trn.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-word_tst.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-word_dev.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-code_trn.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-code_tst.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-code_dev.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-word_trn.json' mode='w' encoding

# Sub-sampling for the multilingual model
In order to have the same amount of training data compared with the monolingual models, we sample 10000 forms for the training of 5lang-sample.
+ we sample from 5 languages 2000 forms and 200 form for training and dev set respectively.
+ we sample 100, 200, 400, 600 tables from the given SIGMORPHON-UniMorph 2023 shared task data
+ we sample from the tables then the forms, keeping the forms per lemma together

In [22]:
from random import sample,shuffle
from math import floor
from itertools import groupby

def get_lemma(line):
    lemma = line.split("\t")[0]
    return lemma

def to_json_format (line, prefix):
    lemma, features, target = line.split("\t")
    json_line = f'{{"input": "{prefix}Inflect {lemma} | {features}","target": "{target}"}}'
    return json_line
    
LANGS = {"nav":"Navajo", "deu":"German", "klr":"Khaling", "amh":"Amharic", "grc":"Ancient Greek"}
PATH_DATA = "../../2023InflectionST/part1/data/"
OUTDIR = "data/"
SPLIT_DATASETS = [".tst",".trn",".dev"]
target_lang = "5lang-sample" # sample from the 5 languages with language name as prefix
with open (OUTDIR + target_lang + "_trn.json", "w") as trn_out, open (OUTDIR + target_lang + "_tst.json", "w") as tst_out, open (OUTDIR + target_lang + "_dev.json", "w") as dev_out:
    for lang in LANGS.keys():
            with open (PATH_DATA + lang + ".trn","r") as train_data, open (PATH_DATA + lang + ".dev","r") as dev_data,open (PATH_DATA + lang + ".tst","r") as test_data:
                data_lines = [line.strip() for line in train_data.readlines() + dev_data.readlines() + test_data.readlines()]

                tables = [(k, list(g)) for k, g in groupby(data_lines, get_lemma) if k != ""]
                
                for n in (500/len(LANGS) ,1000/len(LANGS) ,2000/len(LANGS) ,3000/len(LANGS) ):
                    sampled_tables = sample(tables,int(n))
                    size = len(sampled_tables)
                    datasets = [ sampled_tables[:floor(0.8*size)], sampled_tables[floor(0.8*size):floor(0.9*size)], sampled_tables[floor(0.9*size):] ]
    
                    train_forms = [form for lemma,table in datasets[0] for form in table]
                    dev_forms = [form for lemma,table in datasets[1] for form in table]
                    test_forms = [form for lemma,table in datasets[2] for form in table]
    
                    if len(dev_forms) < 1000/len(LANGS) or len(test_forms) < 1000/len(LANGS) or len(train_forms) < 10000/len(LANGS):
                        print(f"more tables than {n} needed for lang: {lang} ",len(dev_forms),len(test_forms) ,len(train_forms))
                    else:
                        print(f"{lang} needed {n} tables.")
                        break
                        
    
                # sampling and retaining order of the forms
                indices_train = sorted(sample(range(len(train_forms)), int(10000/len(LANGS)) ))
                indices_dev =  sorted(sample(range(len(dev_forms)), int(1000/len(LANGS)) ))
                indices_test = sorted( sample(range(len(test_forms)), int(1000/len(LANGS)) ))
    
                train_samples = [to_json_format(train_forms[i],LANGS[lang]+": ") for i in indices_train]
                dev_samples = [to_json_format(dev_forms[i],LANGS[lang]+": ") for i in indices_dev]
                test_samples = [to_json_format(test_forms[i],LANGS[lang]+": ") for i in indices_test]

                train_txt = "\n".join(train_samples) + "\n"
                dev_txt = "\n".join(dev_samples) + "\n"
                test_txt = "\n".join(test_samples) + "\n"
    
                trn_out.write(train_txt)
                dev_out.write(dev_txt)
                tst_out.write(test_txt)

nav needed 100.0 tables.
more tables than 100.0 needed for lang: deu  118 127 965
more tables than 200.0 needed for lang: deu  247 247 1924
deu needed 400.0 tables.
more tables than 100.0 needed for lang: klr  245 240 1930
klr needed 200.0 tables.
more tables than 100.0 needed for lang: amh  145 145 953
more tables than 200.0 needed for lang: amh  238 237 1864
amh needed 400.0 tables.
more tables than 100.0 needed for lang: grc  173 243 2036
grc needed 200.0 tables.


In [12]:
%cd ~/morphological-inflection/preprocessing/preprocessing_to_json/
!ls ../../2023InflectionST/part1/data/

/dss/dsshome1/03/ge87wod2/morphological-inflection/preprocessing/preprocessing_to_json
afb.covered.dev  eng.trn		ita.dev		 pol.dev
afb.covered.tst  eng.tst		ita.out		 pol.out
afb.dev		 fin.covered.dev	ita.trn		 pol.trn
afb.out		 fin.covered.tst	ita.tst		 pol.tst
afb.trn		 fin.dev		jap.covered.dev  rus.covered.dev
afb.tst		 fin.out		jap.covered.tst  rus.covered.tst
amh.covered.dev  fin.trn		jap.dev		 rus.dev
amh.covered.tst  fin.tst		jap.out		 rus.out
amh.dev		 fra.covered.dev	jap.trn		 rus.trn
amh.out		 fra.covered.tst	jap.tst		 rus.tst
amh.trn		 fra.dev		kat.covered.dev  san.covered.dev
amh.tst		 fra.out		kat.covered.tst  san.covered.tst
arz.covered.dev  fra.trn		kat.dev		 san.dev
arz.covered.tst  fra.tst		kat.out		 san.out
arz.dev		 grc.covered.dev	kat.trn		 san.trn
arz.out		 grc.covered.tst	kat.tst		 san.tst
arz.trn		 grc.dev		klr.covered.dev  sme.covered.dev
arz.tst		 grc.out		klr.covered.tst  sme.covered.tst
bel.covered.dev  grc.trn		klr.dev		 sme.dev
bel.covered.tst  grc.tst		klr

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
